In this notebook, our goal is to test the performance of the dynamic onnx model ```model-int8-dynamic.onnx``` to check if we can still deploy it on our ASP machine.

# Imports

In [28]:
from __future__ import annotations

import argparse
import json
import os
import re
from pathlib import Path
from typing import Dict, Iterable, List, Tuple

import numpy as np
import onnxruntime as ort
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

if "__file__" in globals():
    ROOT = Path(__file__).resolve().parent
else:
    ROOT = Path.cwd()
    nb_root = ROOT / "Mes_notebooks" / "Model_4_DISTILBERT_quant"
    if nb_root.exists():
        ROOT = nb_root
MODEL_PACKAGE_DIR = ROOT.parent / "Model_4_DISTILBERT" / "distilbert_model_package"
DEFAULT_MODEL_PATH = ROOT / "model-int8-dynamic.onnx"

MAX_LENGTH = 128
BATCH_SIZE = 1  # modèle quantifié exporté avec batch statique ; on force 1 par défaut
VALID_EVAL_LABELS = {"positive", "negative"}

# Preprocessing and loading functions

In [29]:
def normalize_tweet(t: str) -> str:
    return re.sub(r"\s+", " ", t.strip())

def load_label_map() -> Dict[int, str]:
    """Read id2label from the saved HF config if available."""
    cfg_path = MODEL_PACKAGE_DIR / "hf_model" / "config.json"
    if not cfg_path.exists():
        return {0: "negative", 1: "positive"}

    try:
        cfg = json.loads(cfg_path.read_text())
        id2label = cfg.get("id2label") or {}
        mapped = {int(k): str(v).lower() for k, v in id2label.items()}
        return mapped or {0: "negative", 1: "positive"}
    except Exception:
        return {0: "negative", 1: "positive"}

def load_tokenizer():
    return AutoTokenizer.from_pretrained(MODEL_PACKAGE_DIR / "tokenizer", use_fast=True)

def softmax(logits: np.ndarray) -> np.ndarray:
    shifted = logits - logits.max(axis=-1, keepdims=True)
    exp = np.exp(shifted)
    return exp / exp.sum(axis=-1, keepdims=True)

def prepare_ort_inputs(session: ort.InferenceSession, encoded: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    base = encoded["input_ids"]
    ort_inputs: Dict[str, np.ndarray] = {}
    for meta in session.get_inputs():
        name = meta.name
        if name in encoded:
            ort_inputs[name] = encoded[name]
        elif name == "token_type_ids":
            ort_inputs[name] = np.zeros_like(base)
        elif name == "position_ids":
            ort_inputs[name] = np.arange(base.shape[1], dtype=np.int64)[None, :].repeat(base.shape[0], axis=0)
        else:
            raise KeyError(f"Impossible de fournir l'entrée ONNX {name}")
    return ort_inputs


In [ ]:
def run_onnx_inference(
    session: ort.InferenceSession,
    tokenizer,
    texts: List[str],
    max_length: int,
) -> Tuple[np.ndarray, np.ndarray]:
    encoded = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="np",
    )
    ort_inputs = prepare_ort_inputs(session, encoded)
    logits = session.run(None, ort_inputs)[0]
    probs = softmax(logits)
    pred_ids = probs.argmax(axis=-1)
    pred_scores = probs.max(axis=-1)
    return pred_ids, pred_scores


def evaluate_dataset(
    session: ort.InferenceSession,
    tokenizer,
    label_map: Dict[int, str],
    dataset: Iterable[Dict[str, str]],
    batch_size: int,
    max_length: int,
    show_category_accuracy: bool = True,
) -> pd.DataFrame:
    dataset = list(dataset)
    rows = []

    normalized_texts = [normalize_tweet(item["tweet"]) for item in dataset]
    raw_texts = [item["tweet"] for item in dataset]
    expected_labels = [item["label"].lower() for item in dataset]
    categories = [item["category"] for item in dataset]

    for start in range(0, len(dataset), batch_size):
        stop = start + batch_size
        batch_texts = normalized_texts[start:stop]
        pred_ids, pred_scores = run_onnx_inference(session, tokenizer, batch_texts, max_length)

        for idx, score, raw_tweet, expected, category in zip(
            pred_ids,
            pred_scores,
            raw_texts[start:stop],
            expected_labels[start:stop],
            categories[start:stop],
        ):
            predicted_label = label_map.get(int(idx), str(idx))
            rows.append(
                {
                    "category": category,
                    "tweet": raw_tweet,
                    "label_attendue": expected,
                    "prediction": predicted_label,
                    "score": round(float(score), 4),
                    "ok": predicted_label == expected if expected in VALID_EVAL_LABELS else None,
                }
            )

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    filtered = df[df["label_attendue"].isin(VALID_EVAL_LABELS)]
    global_acc = (filtered["label_attendue"] == filtered["prediction"]).mean()
    print(f"\n🎯 Accuracy globale (pos/neg) : {global_acc:.4f}")

    if show_category_accuracy and not filtered.empty:
        print("\n📊 Accuracy par catégorie :")
        cat_acc = (
            filtered.groupby("category")
            .apply(lambda g: (g["label_attendue"] == g["prediction"]).mean())
        )
        print(cat_acc)
    return df


def _infer_static_batch(session: ort.InferenceSession) -> int | None:
    """Detect a fixed batch dimension in the ONNX inputs, if any."""
    for meta in session.get_inputs():
        if not meta.shape:
            continue
        dim0 = meta.shape[0]
        if isinstance(dim0, int) and dim0 > 0:
            return dim0
    return None


def choose_batch_size(session: ort.InferenceSession, requested: int) -> int:
    static_batch = _infer_static_batch(session)
    if static_batch is not None and static_batch != requested:
        print(
            f"⚠️  Le modèle semble figé sur batch={static_batch}. "
            f"Batch demandé={requested} => utilisation batch={static_batch}"
        )
        return static_batch
    return requested

# Testing on tweets

In [ ]:
hard_tweets_dataset = [

    # 1 — Sarcasme negatif
    {"category": "sarcasme", "tweet": "Great job @AirParadis, another ‘on-time’ departure… only 3 hours late. Truly impressive 🙃", "label": "negative"},
    {"category": "sarcasme", "tweet": "Loved waiting at the gate for an eternity. Really, I had nothing better to do today 😂", "label": "negative"},
    {"category": "sarcasme", "tweet": "Oh wow, free water after canceling my flight. The generosity is unreal 😒", "label": "negative"},
    {"category": "sarcasme", "tweet": "Fantastic! My luggage is on holiday in Paris while I'm stuck here. Amazing service!", "label": "negative"},
    {"category": "sarcasme", "tweet": "Thanks @AirParadis for this unforgettable disaster of a flight.", "label": "negative"},

    # 2 — Ambiguïté volontaire (converted to negative or positive clearly)
    {"category": "ambiguite", "tweet": "This flight felt terrible from start to finish.", "label": "negative"},
    {"category": "ambiguite", "tweet": "Honestly, this was a pretty good flight overall.", "label": "positive"},
    {"category": "ambiguite", "tweet": "I expected a lot better than what I got today.", "label": "negative"},
    {"category": "ambiguite", "tweet": "The service was surprisingly pleasant, well done @AirParadis!", "label": "positive"},
    {"category": "ambiguite", "tweet": "I’m definitely not flying with them again after today.", "label": "negative"},

    # 3 — Double négation (converted)
    {"category": "double_neg", "tweet": "I can’t say the flight was bad — it was actually very good.", "label": "positive"},
    {"category": "double_neg", "tweet": "I wouldn’t say I enjoyed the flight… because I didn’t.", "label": "negative"},
    {"category": "double_neg", "tweet": "Not unhappy means nothing, I’m definitely upset with this airline.", "label": "negative"},
    {"category": "double_neg", "tweet": "It wasn’t not comfortable — it was genuinely great!", "label": "positive"},
    {"category": "double_neg", "tweet": "I wouldn’t say I don’t recommend @AirParadis, it's a solid choice.", "label": "positive"},

    # 4 — Mixed feelings complexes (converted)
    {"category": "mixed_complex", "tweet": "The landing was smooth and the staff was incredibly kind. Great flight!", "label": "positive"},
    {"category": "mixed_complex", "tweet": "Terrible turbulence and awful food, I hated the whole experience.", "label": "negative"},
    {"category": "mixed_complex", "tweet": "My kids enjoyed it, but I honestly found it stressful and unpleasant.", "label": "negative"},
    {"category": "mixed_complex", "tweet": "The upgrade was amazing and made the whole trip enjoyable.", "label": "positive"},
    {"category": "mixed_complex", "tweet": "Losing my bag ruined everything today.", "label": "negative"},

    # 5 — Positif qui semble négatif / inverse (converted clearly)
    {"category": "pos_neg", "tweet": "The flight was long but honestly quite enjoyable.", "label": "positive"},
    {"category": "pos_neg", "tweet": "My seat didn’t recline and it hurt my back. Terrible comfort.", "label": "negative"},
    {"category": "pos_neg", "tweet": "The chaos was stressful and I didn’t enjoy this flight at all.", "label": "negative"},
    {"category": "pos_neg", "tweet": "The delay was annoying and ruined my entire schedule.", "label": "negative"},
    {"category": "pos_neg", "tweet": "Surprisingly, the trip was comfortable and relaxing.", "label": "positive"},

    # 6 — Implicite (kept negative)
    {"category": "implicite", "tweet": "I don’t even have the energy to complain anymore.", "label": "negative"},
    {"category": "implicite", "tweet": "It happened again… another terrible experience.", "label": "negative"},
    {"category": "implicite", "tweet": "I’m not mad. I’m extremely disappointed.", "label": "negative"},
    {"category": "implicite", "tweet": "No words. Just a horrible flight.", "label": "negative"},
    {"category": "implicite", "tweet": "Silence says it all — this was awful.", "label": "negative"},

    # 7 — Sarcasme positif (kept positive)
    {"category": "sarcasme_positif", "tweet": "Wow, @AirParadis took off on time. Miracles do happen!", "label": "positive"},
    {"category": "sarcasme_positif", "tweet": "Not getting bumped from my seat today? Incredible!", "label": "positive"},
    {"category": "sarcasme_positif", "tweet": "Flight attendants actually smiled. Impressive!", "label": "positive"},
    {"category": "sarcasme_positif", "tweet": "A full flight without any issues? Amazing!", "label": "positive"},
    {"category": "sarcasme_positif", "tweet": "The plane landed early… surprisingly good job.", "label": "positive"},

    # 8 — Positif en surface mais fond négatif (negative)
    {"category": "pos_surface_neg_fond", "tweet": "I’m ‘thrilled’ that my flight was delayed again. Awful service.", "label": "negative"},
    {"category": "pos_surface_neg_fond", "tweet": "Consistently disappointing, as always.", "label": "negative"},
    {"category": "pos_surface_neg_fond", "tweet": "The announcements were useless and the whole trip was terrible.", "label": "negative"},
    {"category": "pos_surface_neg_fond", "tweet": "Free snack but still not getting home today? Terrible airline.", "label": "negative"},
    {"category": "pos_surface_neg_fond", "tweet": "Great, another delay. This airline is exhausting.", "label": "negative"},
]

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Benchmark du modèle ONNX int8 sur les tweets difficiles.")
    parser.add_argument("--model", type=Path, default=DEFAULT_MODEL_PATH, help="Chemin du modèle ONNX (int8).")
    parser.add_argument("--batch-size", type=int, default=BATCH_SIZE, help="Taille de batch pour l'inférence ONNX.")
    parser.add_argument("--max-length", type=int, default=MAX_LENGTH, help="Longueur max de séquence pour le tokenizer.")
    if "__file__" in globals():
        args = parser.parse_args()
    else:
        args = parser.parse_args(args=[])

    if not args.model.exists():
        raise FileNotFoundError(f"Modèle introuvable: {args.model}")

    os.environ.setdefault("CUDA_VISIBLE_DEVICES", "")

    print(f"Chargement du modèle ONNX: {args.model}")
    session = ort.InferenceSession(
        str(args.model),
        providers=["CPUExecutionProvider"],
    )

    effective_batch = choose_batch_size(session, args.batch_size)

    tokenizer = load_tokenizer()
    label_map = load_label_map()

    print(f"\nÉvaluation sur {len(hard_tweets_dataset)} tweets difficiles...")
    df = evaluate_dataset(
        session=session,
        tokenizer=tokenizer,
        label_map=label_map,
        dataset=hard_tweets_dataset,
        batch_size=effective_batch,
        max_length=args.max_length,
    )
    if not df.empty:
        print("\nRésultats détaillés :")
        print(df)

if __name__ == "__main__":
    main()

Chargement du modèle ONNX: /home/kamel/Openclassroom_projets/P7/Mes_notebooks/Model_4_DISTILBERT_quant/model-int8-dynamic.onnx

Évaluation sur 40 tweets difficiles...

🎯 Accuracy globale (pos/neg) : 0.8250

📊 Accuracy par catégorie :
category
ambiguite               1.0
double_neg              0.6
implicite               1.0
mixed_complex           1.0
pos_neg                 1.0
pos_surface_neg_fond    1.0
sarcasme                0.0
sarcasme_positif        1.0
dtype: float64

Résultats détaillés :
                category                                              tweet  \
0               sarcasme  Great job @AirParadis, another ‘on-time’ depar...   
1               sarcasme  Loved waiting at the gate for an eternity. Rea...   
2               sarcasme  Oh wow, free water after canceling my flight. ...   
3               sarcasme  Fantastic! My luggage is on holiday in Paris w...   
4               sarcasme  Thanks @AirParadis for this unforgettable disa...   
5              ambigu

/tmp/ipykernel_18325/442955671.py:75: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["label_attendue"] == g["prediction"]).mean())


# Testing on our dataset

We will reproduce the same test dataset used to evaluate the base distilbert model to see if there is a drop in performance or not

In [ ]:
# Load and preprocess data (same split as notebook_04)
df = pd.read_csv(
    "../../sentiment140/training.1600000.processed.noemoticon.csv",
    encoding="latin-1",
    header=None,
    names=["sentiment", "id", "date", "query", "user", "tweet"],
)

df = df[["sentiment", "tweet"]]
df_negatifs = df[df["sentiment"] == 0].sample(20000, random_state=42)
df_positifs = df[df["sentiment"] == 4].sample(20000, random_state=42)
df = pd.concat([df_negatifs, df_positifs]).reset_index(drop=True)
df["sentiment"] = df["sentiment"].replace(4, 1)

texts = df["tweet"].astype(str).tolist()
labels = df["sentiment"].astype("float32").to_numpy()

X_train, X_tmp, y_train, y_tmp = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels,
)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp,
    y_tmp,
    test_size=0.5,
    random_state=42,
    stratify=y_tmp,
)

test_df = pd.DataFrame({"tweet": X_test})
test_df["label"] = np.where(y_test == 1, "positive", "negative")
test_df["label"].value_counts()

# Build dataset for evaluation
test_dataset = [
    {"category": "sentiment140", "tweet": row.tweet, "label": row.label}
    for row in test_df.itertuples(index=False)
]

if not DEFAULT_MODEL_PATH.exists():
    raise FileNotFoundError(f"Modele introuvable: {DEFAULT_MODEL_PATH}")

os.environ.setdefault("CUDA_VISIBLE_DEVICES", "")
session = ort.InferenceSession(
    str(DEFAULT_MODEL_PATH),
    providers=["CPUExecutionProvider"],
)

effective_batch = choose_batch_size(session, BATCH_SIZE)
tokenizer = load_tokenizer()
label_map = load_label_map()

df_eval = evaluate_dataset(
    session=session,
    tokenizer=tokenizer,
    label_map=label_map,
    dataset=test_dataset,
    batch_size=effective_batch,
    max_length=MAX_LENGTH,
    show_category_accuracy=False,
)

positive_label = "positive"
tp = ((df_eval["label_attendue"] == positive_label) & (df_eval["prediction"] == positive_label)).sum()
fp = ((df_eval["label_attendue"] != positive_label) & (df_eval["prediction"] == positive_label)).sum()
fn = ((df_eval["label_attendue"] == positive_label) & (df_eval["prediction"] != positive_label)).sum()
denom = (2 * tp + fp + fn)
f1 = (2 * tp / denom) if denom else 0.0

print(f"F1 score: {f1:.4f}")


🎯 Accuracy globale (pos/neg) : 0.8235
F1 score: 0.8225
